# HM2 : Szukanie modelu z pomocą frameworka AutoMLowego
W notebooku dwa frameworki AutoMLowe zostały wykorzytsane do wyznaczenia modelu, robiącego dobre predykcje dla zbioru 'artificial'.

### Import plików

In [6]:
import pandas as pd

# nazwy plików 
# root oznacza ścieżkę, którą należy odpowiednio zmienić
file_path_test = "root/artificial_test.data"
file_path_train = "root/artificial_train.data"
file_path_labels = "root/artificial_train.labels"

# import pliku z danymi do trenowania
data = pd.read_csv(file_path_train, sep=" ", header = None )
data = data[data.columns[:-1]]

data_labels = pd.read_csv(file_path_labels, header = None)

# import pliku z danymi do robienia predykcji
test = pd.read_csv(file_path_test, sep=" ", header = None)
test = test[test.columns[:-1]]

### Podział zbioru 'data' na treningowy oraz testowy

In [13]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import balanced_accuracy
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split

# podział danych 7:3
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(
        data, data_labels, test_size = test_size, random_state=4)

def scoring_function(estimator):
    predictions = estimator.predict(X_test)
    return balanced_accuracy_score(y_test, predictions)

## Trening AutoSklearn1, AutoSklearn2


## AutoSklearn2

In [52]:
settings = {
  "time_left_for_this_task": 1800,  # 30 minutes
  "seed": 42,
  "metric": balanced_accuracy,
  "n_jobs": 4,
}
askl2 = AutoSklearn2Classifier(
    **settings
)
askl2.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/autosklearn/experimental/selector.py:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, series in prediction.iteritems():


AutoSklearn2Classifier(metric=balanced_accuracy, n_jobs=4,
                       per_run_time_limit=720, seed=42,
                       time_left_for_this_task=1800)

Statystyki podsumowujące

In [53]:
# statystyki podsumowujące
print(askl2.sprint_statistics())

leaderboard = askl2.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)

auto-sklearn results:
  Dataset name: a6b057d7-b4a0-11ee-a0f7-0242ac1c000c
  Metric: balanced_accuracy
  Best validation score: 0.831533
  Number of target algorithm runs: 35
  Number of successful target algorithm runs: 33
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

          rank  ensemble_weight           type      cost    duration
model_id                                                            
8            2             0.02    extra_trees  0.186365  114.714006
9            4             0.84            sgd  0.480786   44.524346
15           1             0.06  random_forest  0.168467  675.849760
17           5             0.04    extra_trees  0.491542   53.625321
19           3             0.04    extra_trees  0.452243   57.378551


### Wynik dla zbioru testowego

In [54]:
scoring_function(askl2)

0.8334592648562158

## AutoSklearn1

In [14]:
settings = {
  "time_left_for_this_task": 1800,  # 30 minutes
  "seed": 42,
  "metric": balanced_accuracy,
  "n_jobs": 4,
}

askl1 = AutoSklearnClassifier(
    **settings,
    resampling_strategy='cv'
)
askl1.fit(X_train, y_train)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      metric=balanced_accuracy, n_jobs=4,
                      per_run_time_limit=720, resampling_strategy='cv', seed=42,
                      time_left_for_this_task=1800)

Statystyki podsumowujące

In [17]:
# statystyki podsumowujące
print(askl1.sprint_statistics())

leaderboard = askl1.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)

auto-sklearn results:
  Dataset name: 53599121-b4b8-11ee-b767-0242ac1c000c
  Metric: balanced_accuracy
  Best validation score: 0.865022
  Number of target algorithm runs: 58
  Number of successful target algorithm runs: 41
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 5

          rank  ensemble_weight                 type      cost    duration
model_id                                                                  
3            3             0.04        random_forest  0.142860  178.211706
8            2             0.10        random_forest  0.142825  224.459829
9            1             0.20          extra_trees  0.134978  208.847505
20           8             0.06             adaboost  0.374109  403.026729
21          10             0.02   passive_aggressive  0.436448   25.757276
24           7             0.02             adaboost  0.365543  113.137676
31  

Wypisanie operacji wykonanych na zbiorze przed uczeniem przez każdy z modeli wraz z ich wagami.

In [24]:
for i, (weight, pipeline) in enumerate(askl1.get_models_with_weights()):
    print(weight)
    for stage_name, component in pipeline.named_steps.items():
        if "feature_preprocessor" in stage_name:
          print("\nfeature preprocessor\n")
          print(component.choice )
        if "data_preprocessor" in stage_name:
          print("\ndata preprocessor\n")
          print(component.choice)
        if "classifier" in stage_name:
          print("\nclassifier\n")
          print(component.choice)

0.28

data preprocessor

autosklearn.pipeline Feature Type Splitter

feature preprocessor

autosklearn.pipeline Extra Trees Classifier Preprocessing

classifier

autosklearn.pipeline Bernoulli Naive Bayes classifier
0.2

data preprocessor

autosklearn.pipeline Feature Type Splitter

feature preprocessor

autosklearn.pipeline Extra Trees Classifier Preprocessing

classifier

autosklearn.pipeline Extra Trees Classifier
0.1

data preprocessor

autosklearn.pipeline Feature Type Splitter

feature preprocessor

autosklearn.pipeline Extra Trees Classifier Preprocessing

classifier

autosklearn.pipeline Random Forest Classifier
0.08

data preprocessor

autosklearn.pipeline Feature Type Splitter

feature preprocessor

autosklearn.pipeline Extra Trees Classifier Preprocessing

classifier

autosklearn.pipeline K-Nearest Neighbor Classification
0.08

data preprocessor

autosklearn.pipeline Feature Type Splitter

feature preprocessor

autosklearn.pipeline Principle Component Analysis

classifier

a

### Wynik dla zbioru testowego

In [20]:
scoring_function(askl1)

0.8699497755455798

### Szukane predykcje oraz ich zapis do pliku txt wykonane przez model opracowany przez AutoSklearnClassifier

In [21]:
# numery indeksów dodawane ręcznie po utworzeniu pliku
pred = askl1.predict_proba(test)[:,1]
with open("output_skl1.txt", "w") as txt_file:
    for line in pred:
        txt_file.write(str(line)+'\n')